# Decision Tree (Дерево решений)

Рассмотрим на примере конкретной задачи.

**Выиграет ли `Зенит` свой следующий матч?**
**Параметры**:
* Выше ли находится соперник по турнирной таблице
* Дома ли играется матч
* Пропускает ли матч кто-то из лидеров команды
* Наличие дождя

Есть информация об исходах нескольких матчей и планируется создать модель, предсказывающую исход следующего матча, параметры которого ещё не встречались.

Что из себя представляет задача?
* Классификация данных
* Аппроксимация заданной функции

Имеется *частично* заданная функция $f$ и хотим понять как она работает на ещё не известных параметрах

На дереве решений есть метки:
* В узлах, не являющихся листьями: атрибуты, по которым случаи различаются
* В листьях: значения целевой функции
* На ребрах: значения атрибута, из которого исходит ребро

Для классификации нового случая, необходимо спуститься по дереву с учетом критериев разделения до листа и выдать соответствующее значение.

**Исходная Таблица**
Как играет "Зенит"

|Соперник|Играем|Лидеры|Дождь|Победа|
|---|---|---|---|---|
|Выше|Дома|На месте|Да|Нет|
|Выше|Дома|На месте|Нет|Да|
|Выше|Дома|Пропускают|Нет|Да|
|Ниже|Дома|Пропускают|Нет|Да|
|Ниже|В гостях|Пропускают|Нет|Нет|
|Ниже|Дома|Пропускают|Да|Да|
|Выше|В гостях|На месте|Да|Нет|
|Ниже|В гостях|На месте|Нет|?|



**Как использовать**:
Соперник = Ниже
Играем = В гостях
Лидеры = На месте
Дождь = Нет
Победа = ?

Спускаемся по дереву, переходя по нужным ребрам и получаем, что этот матч "Зенит" должен проиграть.

Для создания модели `Дерево решений` используем `DecisionTreeClassifier` из библиотеки `sklearn`. У объекта этого класса есть следующие атрибуты: 
    <p><b>class_weight</b> - веса классов. Если не указано, значит классы должны иметь один вес;</p>
    <p><b>criterion</b> - функция для измерения качества разбиения;</p>
    <p><b>max_depth</b> - максимальная глубина дерева;</p>
    <p><b>max_leaf_nodes</b> - количество листовых узлов. Если None, значит число не ограничено;</p>
    <p><b>min_impurity_decrease</b> - узел разбивается, если это разбиение уменьшает ошибку, большую или равную этому значению;</p>
    <p><b>min_impurity_split</b> - порог для ранней остановки увеличения дерева;</p>
    <p><b>min_samples_leaf</b> - минимальное количество элементов выборки, которые должны быть у листового узла;<p>
    <p><b>min_samples_split</b> - минимальное количество выборок, необходимых для разделения внутреннего узла;</p>
    <p><b>min_weight_fraction_leaf</b> - какая минимальная взвешенная доля суммарного веса должна быть у листового узла;</p>
    <p><b>presort</b> - сортировать ли предварительно данные для ускорения поиска лучших разбиений при подгонке;</p>
    <p><b>random_state</b> - генератор случайных чисел;</p>
    <p><b>splitter</b> - выбор разбиения на каждом узле.</p>

### Датасет
Рассматривать задачу будем на примере известного датасета **Цветки Ириса**

Датасет [Цветки Ириса](https://archive.ics.uci.edu/ml/datasets/iris) содержит 150 записей, каждая из записей содержит 4 признака, т.е. $\boldsymbol x \in \mathbb{R}^4$. 

Что за 4 признака?

0. длина чашелистника, см
1. ширина чашелистника, см
2. длина лепестка, см 
3. ширина лепестка, см 

Т.к. мы говорим про задачу классификации, то какой же физический смысл у классов?

0. Iris Setosa
1. Iris Versicolour 
2. Iris Virginica


## 0. Импорт библиотек

In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import pickle

from sklearn.model_selection import GridSearchCV

## 1. Загружаем данные по цветкам ирисов

Для этого воспользуемся встроенным в библиотеке scikit-learn модулем datasets

In [ ]:
iris = datasets.load_iris()

In [ ]:
# Информация по признакам
iris.data

In [ ]:
# Информация по целевой переменной (классам цветка)
iris.target

In [ ]:
# Выведем информацию по размерности датасета и целевой переменной
# чтобы убедиться, что размерности совпадают
print('Размерность признакового пространства {}'.format(iris.data.shape))
print('Размерность вектора целевой переменной {}'.format(iris.target.shape))

In [ ]:
# Вынесем признаки и целевую перемнную в отдельные переменные
X = iris.data[:, :4] 
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

## 2. Визуально изобразим данные

In [ ]:
from matplotlib.colors import ListedColormap
cmap_bold = ListedColormap(['#FF0000',  '#00FF00', '#0000FF'])

K = 3
x = X[-1]

fig, ax = plt.subplots(figsize=(7,7))
for i, iris_class in enumerate(['Setosa', 'Versicolour', 'Virginica']):
    idx = y==i
    ax.scatter(X[idx,0], X[idx,2], 
               c=cmap_bold.colors[i], edgecolor='k', 
               s=20, label=iris_class);

ax.set(xlabel='длина чашелистика, см', ylabel='длина лепестка, см')
ax.legend();

## 3. Обучим модели дерево решений

### 3.1. Дерево решений с критерием "Индекс Джини"

In [ ]:
# Создадим объект Decision Tree
clf_gini = DecisionTreeClassifier(criterion='gini', max_depth=2, random_state=0)

# Обучение модели
clf_gini.fit(X_train, y_train)

# Прогноз на тестовых данных
y_pred_gini = clf_gini.predict(X_test)

In [ ]:
# точность модели на тестовых данных
accuracy = accuracy_score(y_test, y_pred_gini)*100
print('Точность модели на тестовой выборке: ' + str(round(accuracy, 2)) + ' %.')

In [ ]:
# посмотрим на метрики на обучающей выборке
y_pred_train_gini = clf_gini.predict(X_train)

accuracy = accuracy_score(y_train, y_pred_train_gini)*100
print('Точность модели на обучающей выборке: ' + str(round(accuracy, 2)) + ' %.')

### Визуализируем полученное дерево решений

In [ ]:
plt.figure(figsize=(12,8))

tree.plot_tree(clf_gini.fit(X_train, y_train)) 

А теперь давайте визуализируем границы классов. Для этого выберем первые 2 признака для обучения модели и покажем как выглядят границы принятия решения.

In [ ]:
X_2d = X[:,0:2]

X_2d_train, X_2d_test, y_2d_train, y_2d_test = train_test_split(X_2d, y, test_size = 0.3, random_state = 42)

# Создадим объект Decision Tree
clf_gini = DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=0)

# Обучение модели
clf_gini.fit(X_2d_train, y_2d_train)

# Прогноз на тестовых данных
y_pred_gini = clf_gini.predict(X_2d_test)

# Напишем вспомогательную функцию, которая будет возвращать решетку для дальнейшей визуализации.
def get_grid(data):
    x_min, x_max = X_2d_train[:, 0].min()*0.9, X_2d_train[:, 0].max()*1.1
    y_min, y_max = X_2d_train[:, 1].min()*0.9, X_2d_train[:, 1].max()*1.1
    return np.meshgrid(np.arange(x_min, x_max, 0.01), np.arange(y_min, y_max, 0.01))

# немного кода для отображения разделяющей поверхности
xx, yy = get_grid(X_2d_test)
y_pred = clf_gini.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
plt.pcolormesh(xx, yy, y_pred, cmap='autumn')
plt.scatter(X_2d_train[:, 0], X_2d_train[:, 1], 
            c=y_2d_train, s=100, cmap='autumn', 
            edgecolors='black', linewidth=1.5
);

### 3.2. Дерево решений с критерием "Информационная энтропия"

In [ ]:
clf_en = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=0)

# Обучение модели
clf_en.fit(X_train, y_train)

In [ ]:
y_pred_en = clf_en.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred_en)*100
print('Точность модели на тестовой выборке: ' + str(round(accuracy, 2)) + ' %.')

In [ ]:
# посмотрим на метрики на обучающей выборке
y_pred_train_en = clf_en.predict(X_train)

accuracy = accuracy_score(y_train, y_pred_train_en)*100
print('Точность модели на обучающей выборке: ' + str(round(accuracy, 2)) + ' %.')

### Визуализируем работу алгоритма

In [ ]:
plt.figure(figsize=(12,8))

tree.plot_tree(clf_en.fit(X_train, y_train)) 

In [ ]:
X_2d = X[:,0:2]

X_2d_train, X_2d_test, y_2d_train, y_2d_test = train_test_split(X_2d, y, test_size = 0.3, random_state = 42)

# Создадим объект Decision Tree
clf_en = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=0)

# Обучение модели
clf_en.fit(X_2d_train, y_2d_train)

# Прогноз на тестовых данных
y_pred_gini = clf_en.predict(X_2d_test)

# Напишем вспомогательную функцию, которая будет возвращать решетку для дальнейшей визуализации.
def get_grid(data):
    x_min, x_max = X_2d_train[:, 0].min()*0.9, X_2d_train[:, 0].max()*1.1
    y_min, y_max = X_2d_train[:, 1].min()*0.9, X_2d_train[:, 1].max()*1.1
    return np.meshgrid(np.arange(x_min, x_max, 0.01), np.arange(y_min, y_max, 0.01))

# немного кода для отображения разделяющей поверхности
xx, yy = get_grid(X_2d_test)
y_pred = clf_en.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
plt.pcolormesh(xx, yy, y_pred, cmap='autumn')
plt.scatter(X_2d_train[:, 0], X_2d_train[:, 1], 
            c=y_2d_train, s=100, cmap='autumn', 
            edgecolors='black', linewidth=1.5
);

## 4. Выведем отчет о моделях

In [2]:
print(classification_report(y_test, y_pred_en))

NameError: name 'y_test' is not defined

In [3]:
print(classification_report(y_test, y_pred_gini))

NameError: name 'y_test' is not defined

# 5. GridSearchCV

Как и в предыдущих алгоритмах, в решающих деревьях также много гиперпараметров, которые необходимо задать до обучения модели. Для выбора значений этих параметров также можно использовать `GridSearchCV`. Рассмотрим на хрестоматийном примере Титаник. Данные предварительно очищены и подготовлены для работы моделями.

- `Survived` - выжил/не выжил
- `Age` - возраст 
- `Fare` - стоимость билета 
- `C`,`Q` - порт посадки. Если оба значения =0, то третий порт `S` 
- `Family` - наличие семьи на борту
- `Child`, `Female` - пол (М, Ж, ребенок). Если ни `Child` ни `Female`, то `Male`
- `Class_1`, `Class_2` - класс каюты. Если ни `Class_1` ни `Class_2`, тогда `Class_3`

In [ ]:
# Чтение данных
titanic_dataframe = pd.read_pickle('data/titanic_clean.pickle')

In [ ]:
titanic_dataframe.head()

In [ ]:
titanic_dataframe.info()

In [ ]:
titanic_dataframe.describe()

In [ ]:
# количество пропущенных значений
titanic_dataframe.isna().sum()

In [ ]:
X = titanic_dataframe.drop("Survived", axis=1)
y = titanic_dataframe["Survived"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 22)

> # Задание 
>
> При помощи GridSearchCV найти наилучшее сочетание параметров `criterion`: ('gini', 'entropy'), `min_samples_split`: (3,5,8,10), `min_samples_leaf`: (1,3,5,7), `max_depth`: (3,4,5,6,7,8). Значения для DecisionTreeClassifier `max_features`='auto', `random_state`=22

In [ ]:
from sklearn import tree

# получаю классификатор
DTree_clf = tree.DecisionTreeClassifier(random_state=22)
DTree_clf.fit(X_train, y_train)

In [ ]:
# печатаю дерево решений
tree.plot_tree(DTree_clf)

In [ ]:
from sklearn.model_selection import GridSearchCV
# задаем словарь параметров для модели SVM, которые мы хотим варьироать в рамках
# работы GridSearchCV

parameters = {
    'criterion': ['gini', 'entropy'],
    'min_samples_split': [3, 5, 8, 10],
    'min_samples_leaf': [1, 3, 5, 7],
    'max_depth': [3, 4, 5, 6, 7, 8]
}

grid = GridSearchCV(DTree_clf, parameters, cv=3)
# Fit the data to find the best combination of parameters
grid.fit(X_train, y_train)

# Print the best parameters and the best score
print("Best Parameters: ", grid.best_params_)
print("Best Score: ", grid.best_score_)